Lets visualize some data files and see what we can do later...

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy.matlib

### 50Hertz Load

In [9]:
filepath = "../raw_data/load_data/50 Hertz Load"
filenames = os.listdir(filepath)

In [10]:
# drop 2015-2016-2017 because their format differs
oldformat_filenames = filenames[:-3]
newformat_filenames = filenames[-3:]
# split actual and forecast files
actual_filenames = [filename for filename in oldformat_filenames if 'actual' in filename]
forecast_filenames = [filename for filename in oldformat_filenames if 'forecast' in filename]

In [11]:
# Create new dataframe to ease preprocess.
# This new dataframe should be time series

def merge_actual_forecast(actual_df, forecast_df):
    """
    Merges any two timeseries data. Dataframes should have 'data' column.
    params: 
            actual_df: (pd.dataframe) any timeseries data
            forecast_df: (pd.dataframe) any timeseries data
    returns:
            (pd.dataframe) merged dataframe
    """
    actual_df['actual'] = actual_df['data']
    actual_df['forecast'] = forecast_df['data']
    actual_df.drop('data', axis=1, inplace=True)
    return actual_df

def extract_one_year(full_path):
    """
    1.Reads ExcelFile. ExcelFile should have 'Info' and 'template' sheets, but we do not use them dont worry.
        # todo: change 'Info' and 'template' necessity.
    2. Calls extract_one_month function.
    params: 
            full_path: (str)
    returns:
            (pd.dataframe) dataframe of one year.
    """
    xls = pd.ExcelFile(full_path)
    sheet_names = xls.sheet_names
    sheet_names.remove('Info')
    sheet_names.remove('template')
    
    one_year_df = pd.DataFrame()
    for sheet_name in sheet_names:
        one_year_df = one_year_df.append(extract_one_month(xls, sheet_name))
    one_year_df.reset_index(drop=True, inplace=True)
    
    return one_year_df

def extract_one_month(xls, sheet_name):
    """
    1.Parses xls.
    2. Calls extract_one_day function.
    params: 
            xls: (ExcelFile)
            sheet_name: (str)
    returns:
            (pd.dataframe) dataframe of one month.
    """
    raw_month_df = xls.parse(sheet_name, 
                             header=1, skiprows=1,index_col=False)
    _dates = raw_month_df.columns.values[2:]
    
    one_month_df = pd.DataFrame()
    for day_date in _dates:
        one_day_df = extract_one_day(raw_month_df, day_date)    
        one_month_df = one_month_df.append(one_day_df, ignore_index=True)
    one_month_df.reset_index(drop=True, inplace=True)
    
    return one_month_df

def extract_one_day(raw_month_df, day_date):
    """
    1. Remove 'b'and'a' chars from 'from'and 'to' columns.
    2. Replace 'n.v.' with np.nan
    params: 
            raw_month_df: (pd.Dataframe)
            day_date: (datetime.datetime())
    returns:
            (pd.Dataframe) dataframe of one day. Pre_preprocess applied.
    """
    
    _from = raw_month_df['von'].values
    _to = raw_month_df['bis'].values
    day_sample_size = _from.shape[0]
    
    one_day_df = pd.DataFrame()
    one_day_df['date'] = [day_date] * day_sample_size
    one_day_df['from'] = _from
    one_day_df['to']   = _to
    one_day_df['data'] = raw_month_df.loc[:,day_date].values
    
    one_day_df = prepreprocess_day_dataframe(one_day_df)
    return one_day_df

def prepreprocess_day_dataframe(df):
    def apply_each_quarter(row):
        # what is 'b' and 'a'?
#         print(row['from'],row['to'], row['date'])
        row['from'] = row['from'].replace('a','')
        row['to'] = row['to'].replace('a','')
        

        datetime_format = '%Y-%m-%d-%H:%M'
        row['from'] = str(row['date'].date()) + '-' + row['from'] # add date into 'from'
        row['from'] = pd.to_datetime(row['from'], format=datetime_format)
        

        if row['to'] == '24:00': # 24:00 is not an proper hour.
            row['to'] = '00:00'
            row['to'] = str(row['date'].date()) + '-' + row['to']
            row['to'] = pd.to_datetime(row['to'])
            row['to'] += pd.Timedelta(days=1)
        else:
            row['to'] = str(row['date'].date()) + '-' + row['to'] # add date into 'to'
            row['to'] = pd.to_datetime(row['to'], format=datetime_format)
               
                   
        
        # change 'n.v.' with np.nan
        row['data'] = np.nan if row['data'] == 'n.v.' else row['data']
        
        return row
    df = df.apply(apply_each_quarter, axis=1)
    return df

In [25]:
# retrieve 2010-2011-2012-2013-2014
print("Processing old format files...")
oldformat_dataframe = pd.DataFrame()
for actual_filename, forecast_filename in zip(actual_filenames, forecast_filenames):
    print("Extracting {}...".format(actual_filename))
    actual_year = extract_one_year(os.path.join(filepath, actual_filename))
    print("Extracting {}...".format(forecast_filename))
    forecast_year = extract_one_year(os.path.join(filepath, forecast_filename))
    oldformat_dataframe = oldformat_dataframe.append(merge_actual_forecast(actual_year, forecast_year))
    
oldformat_dataframe.reset_index(drop=True, inplace=True)

# retrieve 2015-2016-2017
print("Processing new format files...")
newformat_dataframe = pd.DataFrame()
for filename in newformat_filenames:
    print("Extracting {}...".format(filename))
    raw_data = pd.read_csv(os.path.join(filepath, filename))
    raw_data.columns = ['date', 'forecast', 'actual']
    # change dtype to datetime
    raw_data['from'] = raw_data['date'].apply(lambda x: pd.to_datetime(x.split(' - ')[0], format='%d.%m.%Y %H:%M'))
    raw_data['to']   = raw_data['date'].apply(lambda x: pd.to_datetime(x.split(' - ')[1], format='%d.%m.%Y %H:%M'))
    raw_data['date'] = raw_data['date'].apply(lambda x: pd.to_datetime(x.split(' ')[0], format='%d.%m.%Y'))
    
    
    raw_data = raw_data[['date','from','to','actual','forecast']]
    newformat_dataframe = newformat_dataframe.append(raw_data)
newformat_dataframe.reset_index(drop=True, inplace=True)

# merge oldformat and newformat into one big file
one_for_all_dataframe = oldformat_dataframe.append(newformat_dataframe)

one_for_all_dataframe.to_csv(os.path.join("../input","Load_50Hertz_2010-2017_cleaned.csv"), index=None)





Processing old format files...
Extracting 2010_actual.xls...
Extracting 2010_forecast.xls...
Extracting 2011_actual.xls...
Extracting 2011_forecast.xls...
Extracting 2012_actual.xls...
Extracting 2012_forecast.xls...
Extracting 2013_actual.xls...
Extracting 2013_forecast.xls...
Extracting 2014_actual.xls...
Extracting 2014_forecast.xls...
Processing new format files...
Extracting 2015.csv...
Extracting 2016.csv...
Extracting 2017.csv...


In [6]:
actual_one_year_df = extract_one_year(os.path.join(filepath,filenames[0]))
forecast_one_year_xls = extract_one_year(os.path.join(filepath,filenames[1]))

In [7]:
xls = pd.ExcelFile(os.path.join(filepath,filenames[0]))
one_month = extract_one_month(xls, '201401')

XLRDError: No sheet named <'201401'>

In [ ]:
one_month

In [ ]:
one_year_df['actual'] = one_year_df['actual'].apply(lambda x: np.nan if x=='n.v.' else x)

In [ ]:
'13:30bb'.replace('b','')

In [ ]:
t = pd.to_datetime('2014-01-01-00:15', format='%Y-%m-%d-%H:%M')
t

In [ ]:
import datetime
t.day += datetime.timedelta(days=1)

In [19]:
pd.to_datetime('2017-12-31')-pd.to_datetime('2010-01-01')


Timedelta('2921 days 00:00:00')

In [20]:
2922*24*4

280512

In [26]:
h = pd.read_csv('../input/Load_50Hertz_2010-2017_cleaned.csv')
h['from'] = h['from'].apply(lambda x: pd.to_datetime(x))
h.loc[(h['from'].diff(1) != pd.Timedelta(minutes=15))]

,date,from,to,actual,forecast
0,2010-01-01 00:00:00,2010-01-01,2010-01-01 00:15:00,6382.0,7275.0


In [23]:
h.iloc[203810:203830]

,date,from,to,actual,forecast
203810,2015-10-25 00:00:00,2015-10-25 00:30:00,2015-10-25 00:45:00,6983.0,5943.0
203811,2015-10-25 00:00:00,2015-10-25 00:45:00,2015-10-25 01:00:00,6789.0,5861.0
203812,2015-10-25 00:00:00,2015-10-25 01:00:00,2015-10-25 01:15:00,6789.0,5800.0
203813,2015-10-25 00:00:00,2015-10-25 01:15:00,2015-10-25 01:30:00,6606.0,5744.0
203814,2015-10-25 00:00:00,2015-10-25 01:30:00,2015-10-25 01:45:00,6634.0,5691.0
203815,2015-10-25 00:00:00,2015-10-25 01:45:00,2015-10-25 02:00:00,6521.0,5654.0
203816,2015-10-25 00:00:00,2015-10-25 02:00:00,2015-10-25 02:15:00,6337.0,5610.0
203817,2015-10-25 00:00:00,2015-10-25 02:15:00,2015-10-25 02:30:00,6369.0,5568.0
203818,2015-10-25 00:00:00,2015-10-25 02:30:00,2015-10-25 02:45:00,6440.0,5538.0
203819,2015-10-25 00:00:00,2015-10-25 02:45:00,2015-10-25 03:00:00,6387.0,5529.0


In [24]:
h.shape

(280524, 5)

In [ ]:
one_for_all_dataframe.plot(['actual','forecast'])